In [113]:
import numpy as np

from itertools import combinations

def get_combinations(arr, k):
    return list(combinations(arr, k))

# HoI at time tf (local complex simplexes)
def get_hoi_values(df_ts, order, t0, tf, col="position_x"):
    m = ((df_ts["time"] >= t0) & (df_ts["time"] <= tf))
    ids = df_ts[m]["permuted_id"].unique()
    size = len(df_ts[m]["time"].unique())
    combinations = get_combinations(arr=ids, k=order)

    df = df_ts[m].pivot(index="time", columns="permuted_id", values=col)
    df.columns = ["".join("{}_{}".format(col, str(id_))).strip() for id_ in df.columns.values]
    ts = df.values.T
    
    df = []
    for c in combinations:
        signs_ = np.zeros(size, dtype="float64")
        values_ = np.ones(size, dtype="float64")
        for k in c:
            values_ *= (ts[k] - np.mean(ts[k])) / np.std(ts[k])
            signs_ += np.sign(values_)

        df_aux = pd.DataFrame({
            "t0": np.repeat(str(t0), size),
            "tf": np.repeat(str(tf), size),
            "time": df_ts[m]["time"].unique(),
            "particles": np.repeat(len(ids), size),
            "order": np.repeat(order, size),
            "column": np.repeat(col, size),
            "combination": np.repeat(str(c), size),
            "cofluctuation": values_
        })

        # Z score
        df_aux["z_cofluctuation"] = df_aux[["cofluctuation"]].apply(lambda x: (x - np.mean(x)) / np.std(x))

        # Coherence rule (1 if it is fully coherent, -1 otherwise)
        # If all the signs are concordant then set the weight sign as positive, otherwise negative
        values_ = df_aux["z_cofluctuation"].values
        exponent = np.sign(order - np.abs(signs_))
        coherence = (-1)**exponent
        df_aux["weight"] = np.where(coherence==1, np.abs(values_), -np.abs(values_))
        
        df.append(df_aux)

    df = pd.concat(df, ignore_index=True)
    return df

def create_simplicial_complex(df_ts, t0, tf, t_simplex, col="position_x", orders=[2, 3]):
    # Estimate HoI for many orders
    df_final = []
    for order in orders:
        df = get_hoi_values(df_ts=df_ts, order=order, t0=t0, tf=tf, col=col)
        df_final.append(df)

    df_final = pd.concat(df_final, ignore_index=True)
    df_final = df_final[df_final["time"] == t_simplex].sort_values(["weight", "order"], ascending=[False, True])
    
    # Fix violations and construct filtration
    
    
    return df_final

# Function that removes all the violating triangles to create a proper filtration
def fix_violations(self, list_simplices, t_current):
    # Sorting the simplices in a descending order according to weights
    sorted_simplices = sorted(
        list_simplices, key=lambda x: x[1], reverse=True)

    # Remove the violations
    list_violating_triangles = []
    list_simplices_for_filtration = []
    set_simplices = set()
    counter = 0
    triangles_count = 0
    violation_triangles = 0
    violation_triangles_negativeterms = 0

    CC_triangles_positive = 0
    total_CC_triangles = 0
    CC_triangles_negative = 0

    # List all the valid simplices that will be
    # used for the computation of the scaffold
    list_simplices_scaffold_all = OrderedDict()
    counter_simplices_all = 0

    # Loop over the sorted simplices, and flipping the sign of all the weights (so that the points in the persistence diagram are above the diagonal)
    for index, i in enumerate(sorted_simplices):
        simplices, weight = i

        # If the current simplex is an edge or a node, then I will immediately include it
        if len(simplices) <= 2:
            # If it's a node I add it in the list of all simplices
            if len(simplices) == 1:
                list_simplices_scaffold_all[str(list(simplices))] = [str(
                    counter_simplices_all), str(-weight)]
            else:
                # If the simplex is an edge, check whether the weight of the last inserted element
                # is different from the current one, if not use the same order idx of appearance (counter_simplices_all)
                if weight != sorted_simplices[index - 1][1]:
                    counter_simplices_all += 1
                    list_simplices_scaffold_all[str(list(simplices))] = [str(
                        counter_simplices_all), str(-weight)]
                else:
                    list_simplices_scaffold_all[str(list(simplices))] = [str(
                        counter_simplices_all), str(-weight)]

            list_simplices_for_filtration.append((simplices, -weight))
            set_simplices.add(tuple(simplices))
            counter += 1
        else:
            # If the current simplex is a triplet, I check whether all the sub-simplices have been included.
            flag = 0
            for t in itertools.combinations(simplices, 2):
                if t in set_simplices:
                    flag += 1

            # If all the sub-simplices already belong to the set, then I add it in the filtration
            if flag == 3:
                set_simplices.add(tuple(simplices))
                list_simplices_for_filtration.append((simplices, -weight))
                counter += 1
                if weight != sorted_simplices[index - 1][1]:
                    counter_simplices_all += 1
                    list_simplices_scaffold_all[str(list(simplices))] = [str(
                        counter_simplices_all), str(-weight)]
                else:
                    list_simplices_scaffold_all[str(list(simplices))] = [str(
                        counter_simplices_all), str(-weight)]

                # Count the number of positive triangles that are in the filtration
                if weight >= 0:
                    triangles_count += 1
            else:
                # Count the violations only for fully coherent state (--- or +++)
                if weight >= 0:
                    violation_triangles += 1
                    list_violating_triangles.append(
                        (simplices, np.abs(weight), 3 - flag))

                else:
                    violation_triangles_negativeterms += 1

    # Fraction of positive triangle discarderd (a.k.a. the hyper coherence)
    hyper_coherence = (1.0 * violation_triangles) / \
        (triangles_count + violation_triangles)
    list_simplices_scaffold_all = dict(list_simplices_scaffold_all)

    return(list_simplices_for_filtration, list_violating_triangles, hyper_coherence, list_simplices_scaffold_all)

# Create Simplexes structure (each node has a weight)
# Filter Simplex
# Estimate persistence (Ripser for higher order)

# A = np.arange(5)  # [1, 2, 3, 4, 5]
# k = 2
# combinations = get_combinations(A, k)

# for c in combinations:
#     for i in c:
#         print("length:", len(c), "c:", str(c), "i", i)
#         # print(i)
# print(combinations_result)

In [103]:
import pandas as pd

df = pd.read_csv("interpolated_23_05_23_3_1080.csv", low_memory=False)
df_final = get_hoi_values(df_ts=df, order=2, t0=0, tf=36000, col="position_x")
df_final#.head(60)

,t0,tf,time,particles,order,column,combination,cofluctuation,z_cofluctuation,weight
0,0,36000,0,4,2,position_x,"(0, 1)",0.224102,0.092040,-0.092040
1,0,36000,1,4,2,position_x,"(0, 1)",0.226022,0.093778,-0.093778
2,0,36000,2,4,2,position_x,"(0, 1)",0.232299,0.099462,-0.099462
3,0,36000,3,4,2,position_x,"(0, 1)",0.232403,0.099556,-0.099556
4,0,36000,4,4,2,position_x,"(0, 1)",0.267115,0.130983,-0.130983
...,...,...,...,...,...,...,...,...,...,...
215965,0,36000,35996,4,2,position_x,"(2, 3)",0.545120,0.512378,-0.512378
215966,0,36000,35997,4,2,position_x,"(2, 3)",0.544911,0.512174,-0.512174
215967,0,36000,35998,4,2,position_x,"(2, 3)",0.544962,0.512224,-0.512224
215968,0,36000,35999,4,2,position_x,"(2, 3)",0.547862,0.515056,-0.515056


In [115]:
df_ = create_simplicial_complex(df_ts=df, t0=0, tf=36000, t_simplex=230, col="position_x", orders=[2, 3, 4])
df_

,t0,tf,time,particles,order,column,combination,cofluctuation,z_cofluctuation,weight
180205,0,36000,230,4,2,position_x,"(2, 3)",-4.232263,-4.152495,4.152495
144210,0,36000,230,4,2,position_x,"(1, 3)",-2.412531,-2.327622,2.327622
72220,0,36000,230,4,2,position_x,"(0, 3)",-2.868332,-2.054661,2.054661
230,0,36000,230,4,2,position_x,"(0, 1)",0.674684,0.499982,-0.499982
108215,0,36000,230,4,2,position_x,"(1, 2)",0.995505,0.520716,-0.520716
216200,0,36000,230,4,3,position_x,"(0, 1, 2)",-0.891604,-0.712489,-0.712489
36225,0,36000,230,4,2,position_x,"(0, 2)",1.183586,1.393048,-1.393048
252195,0,36000,230,4,3,position_x,"(0, 1, 3)",2.160734,1.826818,-1.826818
360180,0,36000,230,4,4,position_x,"(0, 1, 2, 3)",-2.855439,-1.948425,-1.948425
288190,0,36000,230,4,3,position_x,"(0, 2, 3)",3.790540,2.493373,-2.493373


In [120]:
create_simplicial_complex(df_ts=df, t0=0, tf=36000, t_simplex=320, col="position_x", orders=[2, 3, 4])


,t0,tf,time,particles,order,column,combination,cofluctuation,z_cofluctuation,weight
180295,0,36000,320,4,2,position_x,"(2, 3)",-3.149521,-3.095252,3.095252
144300,0,36000,320,4,2,position_x,"(1, 3)",-2.194818,-2.105505,2.105505
72310,0,36000,320,4,2,position_x,"(0, 3)",-2.141635,-1.521119,1.521119
320,0,36000,320,4,2,position_x,"(0, 1)",0.837281,0.647192,-0.647192
108305,0,36000,320,4,2,position_x,"(1, 2)",1.231318,0.800347,-0.800347
216290,0,36000,320,4,3,position_x,"(0, 1, 2)",-1.112959,-0.931191,-0.931191
36315,0,36000,320,4,2,position_x,"(0, 2)",1.201481,1.411967,-1.411967
252285,0,36000,320,4,3,position_x,"(0, 1, 3)",1.983844,1.692241,-1.692241
360270,0,36000,320,4,4,position_x,"(0, 1, 2, 3)",-2.637033,-1.801983,-1.801983
288280,0,36000,320,4,3,position_x,"(0, 2, 3)",2.846777,1.875858,-1.875858
